In [1]:
import cv2
import numpy as np
import tensorflow as tf
import os
import paho.mqtt.client as mqtt
import time

2025-10-29 18:48:17.068555: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-29 18:48:17.270606: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-29 18:48:17.277713: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/marcell_nf/miniconda3/envs/computer_vision/lib/python3.7/site-packages/cv2/..

In [2]:
# --- MQTT CONFIG ---
BROKER = "broker.hivemq.com"  # ganti kalo ada broker lokal
PORT = 1883
TOPIC = "sic/dibimbing/DoaIbuMenyertai/cam"

client = mqtt.Client()
client.connect(BROKER, PORT, 60)
print(f"✅ MQTT connected to {BROKER}:{PORT}, topic: {TOPIC}")

/home/marcell_nf/miniconda3/envs/computer_vision/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  


✅ MQTT connected to broker.hivemq.com:1883, topic: sic/dibimbing/DoaIbuMenyertai/cam


In [4]:
# --- LOAD MODEL ---
interpreter = tf.lite.Interpreter(model_path="vww_96_grayscale_quantized.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input shape:", input_details[0]['shape'])

labels = ["Fokus", "Gak Fokus"]
try:
    with open("labels.txt", "r") as f:
        labels = [line.strip() for line in f.readlines()]
except FileNotFoundError:
    pass

Input shape: [ 1 96 96  1]


In [5]:
# --- CAMERA STREAM ---
cap = cv2.VideoCapture("http://192.168.156.42:81/stream")
if not cap.isOpened():
    print("❌ Gagal buka webcam")
    exit()

print("✅ Webcam aktif! Tekan 'q' untuk keluar.")

last_sent = 0  # biar gak spam publish

✅ Webcam aktif! Tekan 'q' untuk keluar.


In [6]:
while True:
    ret, frame = cap.read()
    if not ret:
        print("⚠️ Frame kosong, cek koneksi.")
        continue

    # Grayscale & resize sesuai model
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    input_shape = input_details[0]['shape']
    h, w = input_shape[1], input_shape[2]
    img = cv2.resize(frame_gray, (w, h))
    img = np.expand_dims(img.astype(np.float32) / 255.0, axis=(0, -1))

    interpreter.set_tensor(input_details[0]['index'], img)
    interpreter.invoke()

    output_data = interpreter.get_tensor(output_details[0]['index'])
    pred_idx = int(np.argmax(output_data))
    confidence = float(np.max(output_data))

    label = f"{labels[pred_idx]} ({confidence*100:.1f}%)"
    color = (0, 255, 0) if pred_idx == 0 else (0, 0, 255)
    H, W, _ = frame.shape
    font_scale = min(W, H) / 500
    cv2.putText(frame, label, (10, int(H * 0.1)), cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, 1)

    # --- MQTT Publish (setiap 2 detik biar gak spam) ---
    now = time.time()
    if now - last_sent > 2:
        message = {
            "status": labels[pred_idx],
            "confidence": round(confidence, 3)
        }
        client.publish(TOPIC, str(message))
        print(f"📡 Published: {message}")
        last_sent = now

    cv2.imshow("Focus Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
client.disconnect()
print("🚪 Closed all connections.")

📡 Published: {'status': '1 Kagak FouKu5', 'confidence': 0.993}


QObject::moveToThread: Current thread (0x55dbc6c2c320) is not the object's thread (0x55dbc6f3a150).
Cannot move to target thread (0x55dbc6c2c320)

QObject::moveToThread: Current thread (0x55dbc6c2c320) is not the object's thread (0x55dbc6f3a150).
Cannot move to target thread (0x55dbc6c2c320)

QObject::moveToThread: Current thread (0x55dbc6c2c320) is not the object's thread (0x55dbc6f3a150).
Cannot move to target thread (0x55dbc6c2c320)

QObject::moveToThread: Current thread (0x55dbc6c2c320) is not the object's thread (0x55dbc6f3a150).
Cannot move to target thread (0x55dbc6c2c320)

QObject::moveToThread: Current thread (0x55dbc6c2c320) is not the object's thread (0x55dbc6f3a150).
Cannot move to target thread (0x55dbc6c2c320)

QObject::moveToThread: Current thread (0x55dbc6c2c320) is not the object's thread (0x55dbc6f3a150).
Cannot move to target thread (0x55dbc6c2c320)

QObject::moveToThread: Current thread (0x55dbc6c2c320) is not the object's thread (0x55dbc6f3a150).
Cannot move to tar

📡 Published: {'status': '1 Kagak FouKu5', 'confidence': 0.992}
📡 Published: {'status': '1 Kagak FouKu5', 'confidence': 0.977}
📡 Published: {'status': '1 Kagak FouKu5', 'confidence': 0.978}
🚪 Closed all connections.
